<a href="https://colab.research.google.com/github/LAnGallego/AlertaEnchente/blob/main/AlertaEnchente.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:

!pip install -q -U google-generativeai



In [94]:
#Configurações iniciais
import google.generativeai as genai
import numpy as np
import pandas as pd

import requests

from google.colab import userdata
api_key = userdata.get('SECRET_KEY')

genai.configure(api_key = api_key)

for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)




models/embedding-001
models/text-embedding-004


In [95]:
title = "A próxima geração de IA para desenvolvedores e google workspace"
sample_text = ("Título: A próxima geração de IA para desenvolvedores e google workspace"
                "\n"
                "Artigo completo: \n"
              "\n"
              "Gemini API Google AI studio: uma maneira acessível de explorar e criar protótipos com aplicações de IA generativa")
embedding = genai.embed_content(model="models/embedding-001",
                                 content=sample_text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")
print(embedding)


{'embedding': [0.047704365, -0.038116436, -0.013336283, 0.015170968, 0.06544785, -0.01089528, -0.024689905, -0.02170119, 0.061980527, 0.056286238, 0.012694725, 0.0053620334, -0.051407944, -0.018484268, 0.0066176937, -0.025848025, 0.01854217, -0.008084727, -0.043392953, -0.0019156138, 0.012856519, 0.0081267245, -0.028519986, -0.057854634, -0.01934927, 0.020270664, 0.02041518, -0.039943915, -0.03970638, 0.021621916, -0.043649495, 0.057142343, -0.026400264, 0.008739, -0.039117165, -0.04514109, -0.02650074, -0.05001597, -0.00081134797, -0.0017510763, 0.0081596775, -0.08677764, -0.00606948, 0.029403245, -0.0048742006, -0.016199743, 0.043065164, 0.041779127, 0.01613166, -0.06010423, 0.02678715, 0.029948542, 0.065747954, -0.04936427, -0.011132892, -0.009787684, 0.028662976, -0.038353287, 0.02273034, 0.00049383787, 0.0027709976, 0.02210621, -0.016863348, 0.052026346, 0.018175397, -0.062278118, -0.044692997, -0.0065305196, 0.014318036, 0.047260553, 0.009355491, 0.011290865, 0.062085375, -0.0593

In [25]:
def  embed_fn(title, text):
  return genai.embed_content(model=model,
                                 content=text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")['embedding'] # originado de {'embedding': [0.047704365, -0.038116436, -0.013336283




In [96]:
def gerar_e_buscar_consulta(consulta, base, model):
  embedding_da_consulta =  genai.embed_content(model=model,
                                 content=consulta,
                                 task_type="RETRIEVAL_QUERY")['embedding'] # originado de {'embedding':...

  produto_escalares = np.dot(np.stack(df['embedding']), embedding_da_consulta )
  indice = np.argmax(produto_escalares)
  return df.iloc[indice]['Conteudo']





In [107]:
import requests

from google.colab import userdata
api_key_w = userdata.get('KEY_WHETHER')

city = 'Porto Alegre'
url = f"http://api.weatherapi.com/v1/current.json?key={api_key_w}&q={city}&aqi=no&lang=pt"

response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    temp_c = data['current']['temp_c']
    condition = data['current']['condition']['text']
    prec = data['current']['precip_mm']
    data_local = data['location']['localtime']
    print(f"data e hora: {data_local}  Temperatura em {city}: {temp_c}°C, {condition}, precipitação de {prec}")
    Tempo = "data e hora:" + data_local + ", Temperatura em " + city + "  "  + str(temp_c) + " °C, "  + "Condição:" + condition + ", precipitação de " + str(prec)

    print(Tempo)
else:
    print("Erro na requisição.")

data e hora: 2024-05-11 20:18  Temperatura em Porto Alegre: 19.0°C, Neblina, precipitação de 0.06
data e hora:2024-05-11 20:18, Temperatura em Porto Alegre  19.0 °C, Condição:Neblina, precipitação de 0.06


In [108]:
DOCUMENT1 = {
    "title": "Previsao",
    "content": Tempo
}

documents = [DOCUMENT1]
df = pd.DataFrame(documents) #tranforama em data frame
df.columns=['Titulo', 'Conteudo']
df

,Titulo,Conteudo
0,Previsao,"data e hora:2024-05-11 20:18, Temperatura em P..."


In [109]:
model='models/embedding-001'
def  embed_fn(title, text):
  return genai.embed_content(model=model,
                                 content=text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")['embedding'] # originado de {'embedding': [0.047704365, -0.038116436, -0.013336283

df['embedding']=df.apply(lambda row: embed_fn(row['Titulo'], row['Conteudo']), axis=1 )
df

,Titulo,Conteudo,embedding
0,Previsao,"data e hora:2024-05-11 20:18, Temperatura em P...","[0.008899483, -0.022088362, -0.060842477, 0.03..."


In [110]:
def gerar_e_buscar_consulta(consulta, base, model):
  embedding_da_consulta =  genai.embed_content(model=model,
                                 content=consulta,
                                 task_type="RETRIEVAL_QUERY")['embedding'] # originado de {'embedding':...

  produto_escalares = np.dot(np.stack(df['embedding']), embedding_da_consulta )
  indice = np.argmax(produto_escalares)
  return df.iloc[indice]['Conteudo']


In [112]:
consulta = 'Temperatura?'

trecho = gerar_e_buscar_consulta(consulta, df, model)
print(trecho)

data e hora:2024-05-11 20:18, Temperatura em Porto Alegre  19.0 °C, Condição:Neblina, precipitação de 0.06
